In [ ]:
# ==============================================
# dashboard_gradio_ingressos.ipynb (script Python)
# ==============================================

import sqlite3
import pandas as pd
import plotly.express as px
import gradio as gr
from sklearn.ensemble import IsolationForest

# 📥 Função de carregar dados de compras de ingressos e aplicar IA
def carregar_dados_ingressos_com_ia():
    # 1️⃣ Abre conexão com o banco e lê apenas as linhas de log que contêm "Compra do ingresso ID"
    conn = sqlite3.connect("../database.db")
    query = """
        SELECT 
            u.id           AS usuario_id,
            u.nome         AS nome,
            u.email        AS email,
            l.acao_usuario AS acao,
            l.data_hora
        FROM logs l
        JOIN usuarios u ON u.id = l.usuario_id
        WHERE l.acao_usuario LIKE 'Compra do ingresso ID%'
    """
    df = pd.read_sql_query(query, conn)
    conn.close()

    # 2️⃣ Converte a coluna data_hora para datetime
    df['data_hora'] = pd.to_datetime(df['data_hora'])

    # --- A PARTIR DAQUI, ALTERAÇÃO PARA BUSCAR “nome_evento” EM VEZ DE “Ingresso X” ---
    # 3️⃣ Extrair o ID numérico do ingresso a partir da string “Compra do ingresso ID …”
    df['ingresso_id'] = (
        df['acao']
        .str.replace("Compra do ingresso ID ", "")  # remove o prefixo textual
        .astype(int)                                # converte para inteiro
    )

    # 4️⃣ Abre nova conexão para ler a tabela de ingressos, trazendo id e nome_evento
    conn2 = sqlite3.connect("../database.db")
    df_ingressos = pd.read_sql_query(
        "SELECT id AS ingresso_id, nome_evento FROM ingressos", conn2
    )
    conn2.close()

    # 5️⃣ Faz o merge entre o DataFrame de logs e a tabela de ingressos
    df = df.merge(df_ingressos, on='ingresso_id', how='left')

    # 6️⃣ Copia o nome_evento para a coluna "evento" (para manter o resto do código idêntico)
    df['evento'] = df['nome_evento']

    # 7️⃣ Mantém a coluna “data” (sem horário) para agrupar mais abaixo
    df['data'] = df['data_hora'].dt.date
    # --- FIM DA ALTERAÇÃO PRINCIPAL ---

    # 8️⃣ Preparar características para a classificação por IsolationForest
    df_features = df.groupby(['usuario_id', 'nome', 'email']).agg(
        total_compras   = ('evento', 'count'),
        dias_diferentes = ('data', pd.Series.nunique)
    ).reset_index()

    modelo = IsolationForest(contamination=0.15, random_state=42)
    df_features['classificacao'] = modelo.fit_predict(
        df_features[['total_compras', 'dias_diferentes']]
    )
    df_features['classificacao'] = df_features['classificacao'].map({1: 'Normal', -1: 'Anômalo'})

    # Junta a classificação de volta ao dataset original
    df = pd.merge(
        df,
        df_features[['usuario_id', 'classificacao']],
        on='usuario_id',
        how='left'
    )

    return df


# 🎯 Função para gerar os gráficos e a saída do dashboard, recebendo filtros opcionais
def gerar_dashboard(email: str = "", nome: str = "", usuario_id: str = ""):
    df = carregar_dados_ingressos_com_ia()

    # 1️⃣ Aplica filtros conforme campos de input (sem alterar essa lógica)
    if email:
        df = df[df['email'].str.contains(email, case=False, na=False)]
    if nome:
        df = df[df['nome'].str.contains(nome, case=False, na=False)]
    if usuario_id:
        try:
            usuario_id_int = int(usuario_id)
            df = df[df['usuario_id'] == usuario_id_int]
        except:
            return "⚠️ ID inválido.", None, None, None, None

    # 2️⃣ Se não encontrou registros, retorna mensagem de aviso
    if df.empty:
        return "⚠️ Nenhuma compra encontrada com os filtros aplicados.", None, None, None, None

    # --- 3️⃣ Mantém toda a lógica de plots exatamente igual ao seu código original ---
    # 3.1️⃣ 🎫 Compras por evento (agora 'evento' já está com o nome real do evento)
    evento_count = df['evento'].value_counts().reset_index()
    evento_count.columns = ['Evento', 'Quantidade']
    fig1 = px.bar(
        evento_count,
        x='Evento',
        y='Quantidade',
        title='🎫 Compras por Evento'
    )

    # 3.2️⃣ 👤 Compras por usuário (com classificação)
    user_count = (
        df.groupby(['nome', 'classificacao'])['evento']
        .count()
        .reset_index(name='Compras')
    )
    fig2 = px.bar(
        user_count,
        x='nome',
        y='Compras',
        color='classificacao',
        title='👤 Compras por Usuário (com Classificação)'
    )

    # 3.3️⃣ 📅 Evolução por data
    data_count = df['data'].value_counts().sort_index().reset_index()
    data_count.columns = ['Data', 'Quantidade']
    data_count['Data'] = data_count['Data'].apply(lambda d: d.strftime('%d/%m/%Y'))
    fig3 = px.line(
        data_count,
        x='Data',
        y='Quantidade',
        markers=True,
        title='📆 Compras por Data'
    )

    # 3.4️⃣ 🥧 Pizza por classificação
        # 3.4️⃣ 🥧 Pizza por classificação
    class_count = df['classificacao'].value_counts().reset_index()
    class_count.columns = ['Classificação', 'Quantidade']

    fig4 = px.pie(
        class_count,
        names='Classificação',
        values='Quantidade',
        color='Classificação',                 # ← diz a Plotly qual coluna usar
        color_discrete_map={                   # ← mapeia cores específicas
            'Anômalo': '#FF0000',              # vermelho para anomalias
            'Normal':  '#636EFA'               # azul-padrão Plotly (ou outra cor que preferir)
        },
        title='🥧 Distribuição das Classificações (IA)'
    )


    # 4️⃣ Retorna exatamente os cinco outputs (status + 4 gráficos), na mesma ordem do original
    return "✅ Dados carregados com sucesso!", fig1, fig2, fig3, fig4


# 🌐 Criação da interface Gradio (SEM NENHUMA ALTERAÇÃO de estrutura/labels, apenas chamando gerar_dashboard)
interface = gr.Interface(
    fn=gerar_dashboard,
    inputs=[
        gr.Textbox(label="📩 Email"),
        gr.Textbox(label="👤 Nome do Usuário"),
        gr.Textbox(label="🆔 ID do Usuário")
    ],
    outputs=[
        gr.Textbox(label="Status"),
        gr.Plot(label="🎫 Compras por Evento"),
        gr.Plot(label="👤 Compras por Usuário"),
        gr.Plot(label="📆 Compras por Data"),
        gr.Plot(label="🥧 Classificação por IA")
    ],
    title="🎟️ Dashboard de Compras de Ingressos com Detecção de Anomalias (IA)",
    description=(
        "Use os filtros para analisar o comportamento de compra "
        "e identificar possíveis anomalias com Isolation Forest."
    )
)

# 🚀 Lança o app (mesma linha que você tinha antes)
if __name__ == "__main__":
    interface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
